In [1]:
import pandas as pd

In [3]:
df=pd.read_csv("md.csv")

C:\Users\tarun\AppData\Local\Temp\ipykernel_33812\138863542.py:1: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("md.csv")


In [9]:
df.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

In [13]:
# Define keywords to filter articles
keywords = ["SARS-CoV-2", "genome", "COVID-19", "mutation", "virus evolution"]

# Filter the data
filtered_data = df[
    df['title'].str.contains('|'.join(keywords), case=False, na=False) |
    df['abstract'].str.contains('|'.join(keywords), case=False, na=False)
]

# Display filtered articles
print(filtered_data[['cord_uid', 'title', 'abstract']].head())


    cord_uid                                              title  \
5   zjufx4fo  Sequence requirements for RNA strand transfer ...   
19  4k8f7ou1  Bioinformatic mapping of AlkB homology domains...   
22  yba7mdtb  Detection and characterization of horizontal t...   
23  bbvxu8op  Comparisons of substitution, insertion and del...   
26  zowp10ts  Recombination Every Day: Abundant Recombinatio...   

                                             abstract  
5   Nidovirus subgenomic mRNAs contain a leader se...  
19  BACKGROUND: AlkB-like proteins are members of ...  
22  Horizontal DNA transfer is an important factor...  
23  Although oligonucleotide probes complementary ...  
26  Viral recombination can dramatically impact ev...  


In [14]:
# Extract articles with available full-text JSON files
full_text_data = filtered_data[filtered_data['pdf_json_files'].notna() | filtered_data['pmc_json_files'].notna()]

# Display relevant columns
print(full_text_data[['cord_uid', 'title', 'pdf_json_files', 'pmc_json_files']].head())


    cord_uid                                              title  \
5   zjufx4fo  Sequence requirements for RNA strand transfer ...   
19  4k8f7ou1  Bioinformatic mapping of AlkB homology domains...   
22  yba7mdtb  Detection and characterization of horizontal t...   
23  bbvxu8op  Comparisons of substitution, insertion and del...   
26  zowp10ts  Recombination Every Day: Abundant Recombinatio...   

                                       pdf_json_files  \
5   document_parses/pdf_json/b2897e1277f56641193a6...   
19  document_parses/pdf_json/83b05e8afa6cbe7a68170...   
22  document_parses/pdf_json/f1d1b9694aa43c837d9b7...   
23  document_parses/pdf_json/867e1b0f6ca8757f2a32a...   
26  document_parses/pdf_json/03203ab50eb64271a9e82...   

                                  pmc_json_files  
5    document_parses/pmc_json/PMC125340.xml.json  
19   document_parses/pmc_json/PMC544882.xml.json  
22   document_parses/pmc_json/PMC546175.xml.json  
23   document_parses/pmc_json/PMC549431.xml.json  

In [ ]:
import json

# Function to load full-text JSON files
def load_full_text(file_path):
    file_path=file_path+
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        # Extract text from sections
        text = " ".join([section['text'] for section in data['body_text']])
        return text
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None

# Example: Load and display full-text content
sample_file = full_text_data['pdf_json_files'].iloc[0].split(';')[0]  # Use the first available file path
full_text = load_full_text(sample_file)
# Display first 1000 characters


In [ ]:
import spacy

# Load a pre-trained model (e.g., SciSpacy for biomedical text)
nlp = spacy.load("en_core_sci_sm")  # Install using "pip install scispacy"

# Process the text
doc = nlp(full_text)

# Extract named entities related to the genome
entities = [(ent.text, ent.label_) for ent in doc.ents]
print(entities)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Example: Create a simple graph
G = nx.Graph()

# Add nodes (entities)
for entity, label in entities:
    G.add_node(entity, label=label)

# Add edges (relationships) manually or based on extracted relationships
# Example edge
G.add_edge("SARS-CoV-2", "genome")

# Visualize the graph
plt.figure(figsize=(10, 6))
nx.draw(G, with_labels=True, node_color='lightblue', edge_color='gray', font_size=10, font_weight='bold')
plt.show()


In [ ]:
# Save filtered data to a CSV file
filtered_data.to_csv("filtered_cord19_data.csv", index=False)

# Save extracted entities to a file
with open("entities.txt", "w") as f:
    for entity, label in entities:
        f.write(f"{entity}\t{label}\n")
